# Simple agent

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream
from llama_index.core.tools import FunctionTool


# for HuggingFace
# from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
# llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
llm = Ollama(model="qwen2.5-coder")

def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the resulting integer"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the resulting integer"""
    return a / b


def add(a: int, b: int) -> int:
    """Adds two integers and returns the resulting integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtracts two integers and returns the resulting integer"""
    return a - b


llm = Ollama(model="qwen2.5-coder")

agent = AgentWorkflow.from_tools_or_functions(
    [FunctionTool.from_defaults(multiply), subtract, multiply, add],
    llm=llm,
    system_prompt="Your are a math agent that can add, subtract ,multiply and divide numbers using provided tools"
)

handler = agent.run("What is (2 + 2) * 2?")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):
        print(ev.delta, end="", flush=True)
        
resp = await handler
resp

Agents are stateless by default, so if we want it to remember previous interactions like in a chatbot manner maintaining context across multiple messages, we need to use a `Context` object.

In [ ]:
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
response